# Apprentissage sur les images

Entraine le modèle sur des images et pas sur les vidéos.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 8
%config InlineBackend.figure_format = 'retina'

## Importation
Les images utilisée pour entrainer le réseau convolutionnel de VGG ont été normalisées.
Il faut appliquer la même transformation à nos images pour conserver les propriétés non-linéaires du réseau de convolution.

In [ ]:
preprocess = ImageDataGenerator(
    preprocessing_function=keras.applications.vgg16.preprocess_input
)

train_dataset = preprocess.flow_from_directory(
    directory='DB/train/',
    classes=None,
    class_mode='categorical',
    batch_size=32,
    target_size=(227, 227)
)

test_dataset = preprocess.flow_from_directory(
    directory="DB/test/",
    classes=None,
    class_mode="categorical",
    batch_size=32,
    target_size=(227, 227)
)

## Modèle de classification
Les images sont
- peu nombreuses
- parfois floues
- une main recouvre en partie les objets
- d'autres objets sont visibles en arrière plan

Le modèle pré-entrainé VGG permet de traiter efficacement les images.
La classification est réduite aux seules 5 classes cibles, le modèle VGG-imagenet complet contient trop de classes et ne parvient pas à distinguer nos 5 objets parmis tous les objets possibles.

In [ ]:
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Input, Flatten, Dense, Dropout

vgg = VGG16(include_top=False, weights="imagenet")
vgg.trainable = False

dense = Sequential(name="dense", [
    Flatten(),
    Dense(512, activation="elu"),
    Dropout(0.1),
    Dense(256),
    Dense(5, activation="softmax")
])

modele = Sequential(name="complet", [
    Input((227, 227, 3)),
    vgg,
    dense
])

dense.summary()

In [ ]:
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy

optimiser = Adam()
modele.compile(optimizer=optimiser, loss=categorical_crossentropy, metrics=["accuracy"])
historique = modele.fit(train_dataset,
                        epochs=2
                       #validation_data=test_dataset
                       )
dense.save("modele-VGG16")